In [2]:
if db_id('Hospital_Py') is null create database Hospital_Py;

Commands completed successfully.

Total execution time: 00:00:00.674

In [1]:
USE [Hospital_Py]

Commands completed successfully.

Total execution time: 00:00:00.872

#### Note: Steps to configure instance (enable spees + clr) and grant SQLRGroup access completed prior. (Ref: [ActionScripts](https://github.com/microsoft/r-server-hospital-length-of-stay/tree/master/Resources/ActionScripts))

## [Step 0](https://microsoft.github.io/r-server-hospital-length-of-stay/dba.html)

##### Create objects, load data, compute stats.

In [34]:
set nocount on;
exec dbo.compute_stats
select * from stats
select * from statsbak

Commands completed successfully.

Total execution time: 00:00:00.411

variable_name,type,mode,mean,std
ClaimClaimStatusID,int,2,NULL,NULL
ClaimStatusDescription,varchar,Closed,NULL,NULL
StatesStateCode,varchar,FL,NULL,NULL
ClaimRoomsWithDamage,smallint,0,1,1.3240661742950535
LossTypeDescription,varchar,Wind,NULL,NULL
PolicyVersionAttributesFormType,varchar,HO3,NULL,NULL
PolicyVersionAttributesOccupancyType,varchar,Owner,NULL,NULL
PolicyVersionAttributesCoverageA,float,NULL,181559.15607599998,245711.62649245863
ClaimMoneyReserve,decimal,NULL,0,0
ClaimMoneyLosses,decimal,NULL,13939.741116,32793.82588103388


variable_name,type,mode,mean,std
rcount,varchar,0,NULL,NULL
gender,varchar,F,NULL,NULL
dialysisrenalendstage,varchar,0,NULL,NULL
asthma,varchar,0,NULL,NULL
irondef,varchar,0,NULL,NULL
pneum,varchar,0,NULL,NULL
substancedependence,varchar,0,NULL,NULL
psychologicaldisordermajor,varchar,0,NULL,NULL
depress,varchar,0,NULL,NULL
psychother,varchar,0,NULL,NULL


In [ ]:
-- runs all steps below. preprocessing, clearnin, feature engineering, training, scoring & evaluating.
-- exec Initial_Run_Once_Py

## [Step 1](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/step1_data_processing.sql)
##### Preprocessing and Cleaning

In [41]:
EXEC [dbo].[fill_NA_mode_mean] @input='LengthOfStay', @output = 'LoS0';

Commands completed successfully.

Total execution time: 00:00:00.110

## [Step 2](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/step2_feature_engineering.sql)
##### Feature Engineering ([Diagrams](https://microsoft.github.io/r-server-hospital-length-of-stay/data-scientist.html))

In [42]:
EXEC [dbo].[feature_engineering]  @input = 'LoS0', @output = 'LoS', @is_production = '0';
EXEC [dbo].[get_column_info] @input = 'LoS';

Commands completed successfully.

Total execution time: 00:00:00.343

## Step 3

##### a. [Splitting](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/step3a_splitting.sql), b. [Training](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/step3b_training.sql), c. [Scoring](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/step3c_scoring.sql), d. [Evaluating](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/step3d_evaluating.sql)

In [20]:
EXEC [dbo].[splitting] @splitting_percent = 70, @input = 'LoS';

(70026 rows affected)

Total execution time: 00:00:00.097

In [ ]:
exec [dbo].[train_model] @model_name = 'RF', @dataset_name = 'LoS'; -- Dev svr Total execution time: 00:00:35.689
select getdate();

In [ ]:
select model_name, (datalength(model)/1024)/1024 as MB  from models where model_name = 'RF';
go
SELECT last_user_update
FROM   sys.dm_db_index_usage_stats us
    JOIN sys.tables t
        ON t.object_id = us.object_id
WHERE  database_id = db_id()
    AND t.object_id = object_id('dbo.models') 

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.130

model_name,MB
RF,2


last_user_update
2021-03-11 19:25:20.960


In [1]:
use Hospital_Py
go
exec sp_rename 'Forest_Prediction', 'Forest_PredictionBak'

Commands completed successfully.

Caution: Changing any part of an object name could break scripts and stored procedures.

Total execution time: 00:00:00.032

In [ ]:
SELECT count(*) FROM LoS WHERE ClaimClaimID NOT IN (SELECT ClaimClaimID FROM Train_Id)

(1 row affected)

Total execution time: 00:00:00.742

(No column name)
29974


In [2]:
EXEC [dbo].[score] @model_name = 'RF',
	@inquery = 'SELECT * FROM LoS WHERE ClaimClaimID NOT IN (SELECT ClaimClaimID FROM Train_Id)',
	@output = 'Forest_Prediction';

STDERR message(s) from external script: 
D:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\revoscalepy\functions\RxDTree.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
 number_ordered_splits = sum([is_ordered.as_matrix()[0][int(x - 1)] for x in (isplit[:, 0])])

STDOUT message(s) from external script: 
Rows Read: 29974, Total Rows Processed: 29974, Total Chunk Time: 1.935 seconds
Total Rows written: 29974, Total time: 0.606

Total execution time: 00:00:09.715

In [6]:
select top 20 * from Forest_Prediction

(20 rows affected)

Total execution time: 00:00:00.033

lengthofstay_Pred,lengthofstay,ClaimClaimID
17.30743474477387,9,420205
19.11814350187633,15,478629
30.97463748640614,40,426634
16.935966505913527,4,421147
17.30743474477387,9,468226
16.935966505913527,4,481943
16.935966505913527,7,429033
16.935966505913527,0,426435
17.60621439720507,12,473187
59.00441803777379,71,470174


### Train GBT, FT, NN

In [ ]:
use Hospital_Py
go
set nocount on;
exec [dbo].[train_model] @model_name = 'GBT', @dataset_name = 'LoS';
go
exec [dbo].[train_model] @model_name = 'FT', @dataset_name = 'LoS'; --failed, set max_num_bins
go
exec [dbo].[train_model] @model_name = 'NN', @dataset_name = 'LoS'; --failed
go


In [1]:
select model_name, (datalength(model)/1024)/1024 as MB from models;

(2 rows affected)

Total execution time: 00:00:00.089

model_name,MB
RF,2
GBT,0


### Score GBT, FT, NN

In [9]:
drop table if exists Boosted_PredictionBak;
exec sp_rename 'Boosted_Prediction', 'Boosted_PredictionBak';
exec [dbo].[score] @model_name = 'GBT',	@inquery = 'SELECT * FROM LoS WHERE ClaimClaimID NOT IN (SELECT ClaimClaimID FROM Train_Id)', @output = 'Boosted_Prediction';
go 
-- drop table if exists Fast_PredictionBak;
-- exec sp_rename 'Fast_Prediction', 'Fast_PredictionBak';
-- exec [dbo].[score] @model_name = 'FT', @inquery = 'SELECT * FROM LoS WHERE ClaimClaimID NOT IN (SELECT ClaimClaimID FROM Train_Id)', @output = 'Fast_Prediction';
-- go
-- drop table if exists NN_PredictionBak;
-- exec sp_rename 'NN_Prediction', 'NN_PredictionBak';
-- exec [dbo].[score] @model_name = 'NN', @inquery = 'SELECT * FROM LoS WHERE ClaimClaimID NOT IN (SELECT ClaimClaimID FROM Train_Id)', @output = 'NN_Prediction';
-- go
SELECT top 20 *  FROM [Hospital_Py].[dbo].[Boosted_Prediction]
-- SELECT top 20 *  FROM [Hospital_Py].[dbo].[Fast_Prediction]
-- SELECT top 20 *  FROM [Hospital_Py].[dbo].[NN_Prediction]


Caution: Changing any part of an object name could break scripts and stored procedures.

STDERR message(s) from external script: 
D:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\revoscalepy\functions\RxDTree.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
 number_ordered_splits = sum([is_ordered.as_matrix()[0][int(x - 1)] for x in (isplit[:, 0])])

STDOUT message(s) from external script: 
Rows Read: 29974, Total Rows Processed: 29974, Total Chunk Time: 1.945 seconds
Total Rows written: 29974, Total time: 0.614

Commands completed successfully.

Total execution time: 00:00:08.761

lengthofstay_Pred,lengthofstay,ClaimClaimID
25.628452103238345,9,420205
25.628452103238345,15,478629
25.628452103238345,40,426634
25.628452103238345,4,421147
25.628452103238345,9,468226
25.628452103238345,4,481943
25.628452103238345,7,429033
25.628452103238345,0,426435
25.628452103238345,12,473187
41.24410339279123,71,470174


# 🥊 Realtime Scoring

#### [Proc](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/partA_train_real_time_scoring_sp.sql) ([errors](https://stackoverflow.com/questions/55643467/dataframe-object-has-no-attribute-str-problem)), [Train](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/partB_prepare_for_real_time_scoring.sql), [Score](https://github.com/microsoft/r-server-hospital-length-of-stay/blob/master/SQLPy/partC_real_time_scoring.sql)

In [3]:
-- My simple proc to serialize the model bin, cause train_model_real_time_scoring errors. 
Use Hospital_Py
go
create or alter proc [GetRTSModelRF]   
as
declare @info varbinary(max);
select @info = info from dbo.ColInfo;
declare @info varbinary(max);
select @info = info from dbo.ColInfo;

exec sp_execute_external_script @language = N'Python', @script = N' 
import dill
from numpy import sqrt
from pandas import DataFrame
from revoscalepy import rx_set_compute_context, RxSqlServerData, rx_dforest, RxOdbcData, rx_serialize_model, rx_write_object, RxLocalSeq
from microsoftml import adadelta_optimizer

connection_string = "Driver=SQL Server;Server=localhost;Database=Hospital_Py;Trusted_Connection=true;"

column_info = dill.loads(info)

##	Set training dataset, set features and types.

variables_all = [var for var in column_info]
#variables_to_remove = ["eid", "vdate", "discharged", "facid"]
variables_to_remove = ["ClaimClaimID", "ClaimDateClosed", "ClaimReportedDate"]
training_variables = [x for x in variables_all if x not in variables_to_remove]
LoS_Train = RxSqlServerData(sql_query = "SELECT ClaimClaimID, {} FROM LoS WHERE ClaimClaimID IN (SELECT ClaimClaimID from Train_Id)".format(", ".join(training_variables)),
                            connection_string = connection_string,
                            column_info = column_info)

##	Specify the variables to keep for the training 

#variables_to_remove = ["eid", "vdate", "discharged", "facid", "lengthofstay"]
variables_to_remove = ["ClaimClaimID", "ClaimDateClosed", "ClaimReportedDate", "lengthofstay"]
training_variables = [x for x in variables_all if x not in variables_to_remove]
formula = "lengthofstay ~ " + " + ".join(training_variables)

## Train RF Model
dest = RxOdbcData(connection_string, table = "RTS")
model = rx_dforest(formula=formula,
                    data=LoS_Train,
                    n_tree=40,
                    cp=0.00005,
                    min_split=int(sqrt(70000)),
                    max_num_bins=int(sqrt(70000)),
                    seed=5)
serialized_model = rx_serialize_model(model, realtime_scoring_only = True)
rx_write_object(dest, key_name="id", key="RF", value_name="value", value=serialized_model, serialize=False, compress=None, overwrite=False)'

, @params = N'@info varbinary(max)'
, @info = @info;

GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.005

In [4]:
-- drop table if exists RTSBak;
-- go
-- exec sp_rename 'RTS', 'RTSBak'
go
exec GetRTSModelRF;

Commands completed successfully.

STDERR message(s) from external script: 
D:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\revoscalepy\functions\RxDTree.py:91: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
 number_ordered_splits = sum([is_ordered.as_matrix()[0][int(x - 1)] for x in (isplit[:, 0])])

STDOUT message(s) from external script: 
Number of observations not available for this data source. 'numObs' set to 1e6.

For this data source, explicitly set 'max_num_bins' and 'min_split', otherwise innacurate models can result.
Please see rx_dtree 'max_num_bins' and 'min_split' for details.

oneFilePerTree is not a logical and is being reset to FALSE

useWriteDataSource is not an integer and is being reset to 0

autoCleanup is not a logical and is being reset to TRUE
oneFilePerTree is not a logical and is being reset to FALSE
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.826 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.676 seconds 

Elapsed time to compute low/high values and/or factor levels: 2.573 secs.
 
The variance is zero for the variable(s): "ClaimClaimStatusID", "ClaimStatusDescription", "ClaimMoneyReserve"
which are eliminated for further analysis.

STDOUT message(s) from external script: 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.795 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.763 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.787 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.690 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.951 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.697 seconds 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: Less than .001 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 2.249 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.736 seconds 
Rows Read: 4, Total Rows Processed: 4, Total Chunk Time: Less than .001 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.870 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.713 seconds

STDOUT message(s) from external script: 
Rows Read: 8, Total Rows Processed: 8, Total Chunk Time: 0.002 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.841 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.678 seconds 
Rows Read: 16, Total Rows Processed: 16, Total Chunk Time: 0.003 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.871 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.677 seconds 
Rows Read: 28, Total Rows Processed: 28, Total Chunk Time: 0.004 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.886 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.706 seconds 
Rows Read: 32, Total Rows Processed: 32, Total Chunk Time: 0.005 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.900 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.721 seconds

...

STDOUT message(s) from external script: 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.762 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.649 seconds 
Rows Read: 56, Total Rows Processed: 56, Total Chunk Time: 0.007 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.788 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.671 seconds 
Rows Read: 66, Total Rows Processed: 66, Total Chunk Time: Less than .001 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.790 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.657 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.866 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.768 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.758 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.631 seconds

STDOUT message(s) from external script: 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: Less than .001 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.772 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.646 seconds 
Rows Read: 4, Total Rows Processed: 4, Total Chunk Time: 0.002 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.764 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.666 seconds 
Rows Read: 8, Total Rows Processed: 8, Total Chunk Time: Less than .001 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.769 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.654 seconds 
Rows Read: 16, Total Rows Processed: 16, Total Chunk Time: 0.002 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.761 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.643 seconds

STDOUT message(s) from external script: 
Rows Read: 30, Total Rows Processed: 30, Total Chunk Time: 0.008 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.789 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.643 seconds 
Rows Read: 50, Total Rows Processed: 50, Total Chunk Time: 0.006 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.757 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.731 seconds 
Rows Read: 64, Total Rows Processed: 64, Total Chunk Time: 0.006 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.801 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.660 seconds 
Rows Read: 62, Total Rows Processed: 62, Total Chunk Time: 0.009 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.807 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.658 seconds

STDOUT message(s) from external script: 
Rows Read: 62, Total Rows Processed: 62, Total Chunk Time: 0.006 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.897 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.679 seconds 
Rows Read: 60, Total Rows Processed: 60, Total Chunk Time: 0.003 seconds 
Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.787 seconds
Rows Read: 20026, Total Rows Processed: 70026, Total Chunk Time: 0.646 seconds 

Elapsed time for DForestEstimation: 1209.523 secs.

Elapsed time for BxDTreeBase: 1212.209 secs.
 
Rows Read: 1, Total Rows Processed: 1
Total Rows written: 1, Total time: 0.014
, Total Chunk Time: 0.031 seconds

Total execution time: 00:20:17.325

In [8]:
select id, (datalength(value)/1024)/1024 as MB from RTS

(1 row affected)

Total execution time: 00:00:00.010

id,MB
RF,0


In [22]:
--- Perform Real Time Scoring on previously trained models
--- Use 'RF' for Random Forest, use 'GBT' for b_trees, use 'FT' for Fast Trees, or use 'NN' for Neural Network
--- Results are written to RTS_Prediction

--- NOTE: Run prepare_real_time_scoring.sql before running this script.

Use Hospital_Py
GO

--- Real Time Scoring

--	Get the trained model
DECLARE @model_name VARCHAR(3) = 'RF'
DECLARE @model VARBINARY(max) = (SELECT value FROM [dbo].[RTS] WHERE id = @model_name);		

--- Real Time Scoring is meant for small scoring request, which is why we select the top 10 for this example.
DECLARE @inputData VARCHAR(max);
SET @inputData = 'SELECT TOP (10) ClaimClaimID, ClaimClaimStatusID, ClaimStatusDescription, StatesStateCode, LossTypeDescription, PolicyVersionAttributesFormType, 
	PolicyVersionAttributesOccupancyType, PolicyVersionAttributesCoverageA, ClaimMoneyLosses, ClaimMoneyLAE, ClaimRoomsWithDamage, 
	number_of_issues, lengthofstay
FROM LoS WHERE 	ClaimClaimID NOT IN (SELECT ClaimClaimID from Train_Id) ORDER BY ClaimClaimID';

DECLARE @output_table TABLE(lengthofstay_Pred FLOAT);
INSERT @output_table EXEC [dbo].[sp_rxPredict] @model = @model, @inputData = @inputData;
DROP TABLE IF EXISTS RTS_PredictionBak;
exec sp_rename 'RTS_Prediction', 'RTS_PredictionBak';
SELECT * INTO RTS_Prediction FROM @output_table

Commands completed successfully.

(10 rows affected)

Caution: Changing any part of an object name could break scripts and stored procedures.

(10 rows affected)

Total execution time: 00:00:00.216

In [ ]:
select top 10 ClaimClaimID, LengthOfStay from LoS Order by ClaimClaimID
select * from RTS_Prediction

(10 rows affected)

(10 rows affected)

Total execution time: 00:00:00.013

ClaimClaimID,LengthOfStay
402607,16
402619,19
402627,25
402647,16
402658,14
402660,16
402663,5
402665,5
402666,5
402667,17


lengthofstay_Pred
24.000814132212433
281.10619473986014
204.47810635851164
53.38754342384557
57.4645801226574
475.1654887397811
239.28146462247355
45.95507795975456
57.4645801226574
305.4491891191269


In [10]:
SELECT TOP 10 * FROM LoS WHERE ClaimClaimID NOT IN (SELECT ClaimClaimID from Train_Id) ORDER BY ClaimClaimID

(10 rows affected)

Total execution time: 00:00:00.121

ClaimClaimID,ClaimReportedDate,ClaimClaimStatusID,ClaimStatusDescription,StatesStateCode,LossTypeDescription,PolicyVersionAttributesFormType,PolicyVersionAttributesOccupancyType,ClaimMoneyReserve,PolicyVersionAttributesCoverageA,ClaimMoneyLosses,ClaimMoneyLAE,ClaimRoomsWithDamage,number_of_issues,ClaimDateClosed,lengthofstay
402660,2018-02-15,2,Closed,FL,Loss Assessment,HO6,Owner,0.00,-0.47437379231860405,-0.4235474435458612,-0.3708139714876188,-0.7552492612632524,0,2018-03-03,16
402663,2018-02-15,2,Closed,FL,Water Damage - Weather Related,HO3,Owner,0.00,0.2791314554507051,0.6279571940982316,0.3609464911468376,-0.7552492612632524,0,2018-02-20,5
402674,2018-02-16,2,Closed,FL,Water Damage - Non Weather Related,HO6,Tenant,0.00,-0.49472284975383884,-0.3411310152277759,-0.3212343237103772,-0.7552492612632524,0,2018-03-15,27
402682,2018-02-16,2,Closed,PA,All Other Physical Damage,HO3,Owner,0.00,1.8914523930281508,-0.29414564653094655,-0.3043935271879434,-0.7552492612632524,0,2018-02-28,12
402685,2018-02-16,2,Closed,FL,Loss Assessment,HO6,Owner,0.00,-0.5354209646243084,-0.3640850311065803,-0.255113842707539,-0.7552492612632524,0,2018-03-01,13
402686,2018-02-16,2,Closed,FL,Water Damage - Non Weather Related,HO3,Owner,0.00,0.48203190713743127,8.956154733195175,7.430288637459035,-0.7552492612632524,0,2020-10-24,981
402688,2018-02-16,2,Closed,FL,Water Damage - Non Weather Related,HO3,Owner,0.00,0.1779885003746141,0.3494639181647901,0.5190271485800652,-0.7552492612632524,0,2019-10-07,598
402690,2018-02-16,2,Closed,MA,Water Damage - Non Weather Related,HO3,Owner,0.00,0.683251526680007,-0.355742302173625,-0.3506735786999753,-0.7552492612632524,0,2018-04-16,59
402691,2018-02-16,2,Closed,FL,Loss Assessment,HO6,Owner,0.00,-0.47437379231860405,-0.3640850311065803,-0.255113842707539,-0.7552492612632524,0,2018-03-06,18
402695,2018-02-16,2,Closed,FL,Wind,HO3,Owner,0.00,-0.06252921888688706,0.7686230016418365,1.5084989350972622,-0.7552492612632524,0,2019-01-19,337


In [19]:
exec do_native_predict 402690

(1 row affected)

(1 row affected)

Elapsed Time for 1 row scoring is : 13333 microseconds.

Total execution time: 00:00:00.053

LengthOfStay_Pred
45.95507795975456


In [ ]:
SELECT * FROM LoS WHERE ClaimClaimID =402695

(1 row affected)

Total execution time: 00:00:00.037

ClaimClaimID,ClaimReportedDate,ClaimClaimStatusID,ClaimStatusDescription,StatesStateCode,LossTypeDescription,PolicyVersionAttributesFormType,PolicyVersionAttributesOccupancyType,ClaimMoneyReserve,PolicyVersionAttributesCoverageA,ClaimMoneyLosses,ClaimMoneyLAE,ClaimRoomsWithDamage,number_of_issues,ClaimDateClosed,lengthofstay
402695,2018-02-16,2,Closed,FL,Wind,HO3,Owner,0.00,-0.06252921888688706,0.7686230016418365,1.5084989350972622,-0.7552492612632524,0,2019-01-19,337
